# Wykrywanie wariantów DNA

na bazie 
https://app.terra.bio/#workspaces/help-gatk/GATKTutorials-Germline-June2019/notebooks/launch/1-gatk-germline-variant-discovery-tutorial.ipynb


### Najpierw troche magii :), czyli kod pozwalający podglądać rozwiązania kolejnych zadań

In [1]:
from __future__ import print_function
from IPython.core.magic import Magics, magics_class, line_magic
import subprocess
def create_new_cell(contents):
    from IPython.core.getipython import get_ipython
    shell = get_ipython()
    shell.set_next_input(contents, replace=True)

def get_solution(solution_file, solution_local):
    subprocess.call(['gsutil' ,'cp', solution_file, solution_local ])
    myfile = open(solution_local)
    lines = myfile.read()
    myfile.close()
    ! rm {solution_local}
    return lines
    


@magics_class
class MyMagics(Magics):

    @line_magic
    def lmagic(self, line):
        "Replace current line with new output"
        raw_code = get_solution("gs://bucket-edugen/"+line,line)
        self.shell.set_next_input('# %lmagic '+line+'\n{}'.format(raw_code), replace=True)
      
ip = get_ipython()
ip.register_magics(MyMagics)

In [2]:
#####
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/gatk-tutorials/workshop_1906/2-germline/images/vd-image1.png")

## Ustawienia zmiennych

In [3]:
import os
user_name = os.environ.get('USER')
print(user_name)

workspace = "/home/jovyan/work/git/edugen_pub/"
os.environ['WORKSPACE'] = workspace
! echo ${WORKSPACE}

tgambin
/home/jovyan/work/git/edugen_pub/


In [4]:
def clean_all(): 
    ! rm -r sandbox_trio
    ! gsutil rm gs://bucket-$USER/*vcf*
    ! gsutil rm gs://bucket-$USER/*bam*

In [5]:
clean_all()

rm: cannot remove 'sandbox_trio': No such file or directory
CommandException: No URLs matched: gs://bucket-tgambin/*vcf*
CommandException: No URLs matched: gs://bucket-tgambin/*bam*


### Utworzenie własnego kubełka i jego upubliczenienie

In [6]:
! gsutil mb gs://bucket-$USER
#! gsutil iam ch -d allUsers:objectViewer gs://bucket-$USER

No changes made to gs://bucket-tgambin/


## Wykrywanie wariantów GATK HaplotypeCaller w trybie standardowym

Wywolanie GATK HC w najprostszej wersji dla pojedynczej próbki

In [8]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam \
    -O ${WORKSPACE}/sandbox/motherHC.vcf \
    -L 20:10,000,000-10,200,000

Using GATK jar /opt/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar HaplotypeCaller -R /home/jovyan/work/git/edugen_pub//ref/ref.fasta -I /home/jovyan/work/git/edugen_pub//bam/mother.bam -O /home/jovyan/work/git/edugen_pub//sandbox/motherHC.vcf -L 20:10,000,000-10,200,000
22:05:55.585 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
22:05:56.330 INFO  HaplotypeCaller - ------------------------------------------------------------
22:05:56.330 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.1.9.0
22:05:56.330 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
22:05:56.330 INFO  HaplotypeCall

## Wizualizacja BAM i VCF w programie IGV

### Skopiowanie do publicznego kubełka

In [9]:
! bgzip -c ${WORKSPACE}/sandbox/motherHC.vcf > ${WORKSPACE}/sandbox/motherHC.vcf.gz
! tabix ${WORKSPACE}/sandbox/motherHC.vcf.gz
! gsutil cp sandbox/motherHC.vcf.gz* gs://bucket-$USER/

Copying file://sandbox/motherHC.vcf.gz [Content-Type=text/vcard]...
Copying file://sandbox/motherHC.vcf.gz.tbi [Content-Type=application/octet-stream]...
- [2 files][ 18.1 KiB/ 18.1 KiB]                                                
Operation completed over 2 objects/18.1 KiB.                                     


<div class="alert alert-block alert-warning">
<b>Zadanie 1:</b> Skopiuj plik bam i bai do swojego kubełka  
</div>

In [ ]:
%lmagic ex1
 # Solution to ex 1
! gsutil cp bam/mother.bam* gs://bucket-$USER/
! gsutil ls gs://bucket-tgambin/

In [ ]:
! zcat ${WORKSPACE}/sandbox/motherHC.vcf.gz | grep 100024

### Uruchomienie IGV

In [ ]:
from igv_jupyterlab import IGV
# At minimum, IGV requires a single argument, genome.
# For supported genomes, a simple name may be supplied.
igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)
t1 = igv.create_track(
        name = "mother BAM",
        url = "gs://bucket-"+user_name+"/mother.bam",
        index_url= "gs://bucket-"+user_name+"/mother.bam.bai",        
        indexed= True
)

t2 = igv.create_track(
        name= "mother HC VCF",
        url= "gs://bucket-"+user_name+"/motherHC.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/motherHC.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)
igv.load_track(t1)
igv.load_track(t2)
igv.search_locus("20", 10002294, 10002623)


<div class="alert alert-block alert-warning">
<b>Zadanie 2:</b> Jaka jest zygotyczność dwóch widocznych wariantów? Czy jest ona zgodna z tym co widzimy w odczytach ? 
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 3:</b> Kliknij na ikonę edycji track'u z odczytami i zaznacz opcję "Soft clip reads". Algorytm mapowania (BWA MEM) zdecydował, że kara za obcięcie odczytów (Soft clipping) będzie mniejsza niż wstawienie insersji/delecji. 
</div>



## Ponowne uliniowienie odczytów i zasemblowane haplotypy

HaplotypeCaller posiada parametr -bamout pozwalający na wyświetlenie ponownie uliniowionych odczytów,
które następnie są wykorzystywane do wykrywania wariantów.


In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam\
    -O ${WORKSPACE}/sandbox/motherHCdebug.vcf \
    -bamout ${WORKSPACE}/sandbox/motherHCdebug.bam \
    -L 20:10,002,000-10,003,000

<div class="alert alert-block alert-warning">
<b>Zadanie 4:</b> Wyświetl dla porównania obydwa BAMy (zwykły i debug) oraz VCF. Przy wyświetlaniu BAM'ów ustaw opcję 
    showSoftClips = True, aby od razu wyświetlały się split ready. Inne opcje są dostępne na: https://github.com/igvteam/igv.js/wiki/Alignment-Track
</div>

In [ ]:
# %lmagic ex4
#Solution to ex4

! gsutil cp sandbox/motherHCdebug.ba* gs://bucket-$USER/
from igv_jupyterlab import IGV
# At minimum, IGV requires a single argument, genome.
# For supported genomes, a simple name may be supplied.
igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)
t1 = igv.create_track(
        name = "mother BAM",
        url = "gs://bucket-"+user_name+"/mother.bam",
        index_url= "gs://bucket-"+user_name+"/mother.bam.bai",    
        showSoftClips=True,
        indexed= True
)

t2 = igv.create_track(
        name= "mother HC debug BAM",
        url= "gs://bucket-"+user_name+"/motherHCdebug.bam",
        index_url= "gs://bucket-"+user_name+"/motherHCdebug.bai",
        indexed= True,
        showSoftClips=True,
        visibilityWindow= 1000
)
t3 = igv.create_track(
        name= "mother HC VCF",
        url= "gs://bucket-"+user_name+"/motherHC.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/motherHC.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)
igv.load_track(t1)
igv.load_track(t2)
igv.load_track(t3)
igv.search_locus("20", 10002294, 10002623)

Ponieważ jesteśmy zainteresowani jednym miejscem (20:10,002,000-10,003,000) to dokonalismy reuliniowienia tylko tego fragmentu. 

Po ponownym uliniowieniu zniknęły obcięte odczyty (soft clipped reads) . HaplotypeCaller wykorzystuje odczyty 'soft-clipped' do ponownego uliniowienia. Z analizy odczytów wynika, że insercja i pobliski SNP są ze sobą sfazowane. Wynika z tego, że HaplotypeCaller znalazł różne możliwe uliniowienia po wykonaniu lokalnej reasemblacji. Po reasemblacji występuje wystarczająco dużo odczytów wskazujących na występowanie insercji, która byłaby pominięta w przypadku użycia prostszych algorytmów, nie wykorzystujących reasemblacji (np. UnifiedGenotyper)

<div class="alert alert-block alert-warning">
<b>Zadanie 5:</b> Przyjrzyj się insercji w poszczególnych odczytach.  Czy wszędzie jest ona taka sama?
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 6:</b> W jaki sposób wyświetlić w IGV haplotypy utworzone przez HC?
</div>

In [ ]:
# %lmagic ex6
#Solution to ex6


igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)


t2 = igv.create_track(
        name= "mother HC debug BAM",
        url= "gs://bucket-"+user_name+"/motherHCdebug.bam",
        index_url= "gs://bucket-"+user_name+"/motherHCdebug.bai",
        indexed= True,
        showSoftClips=True,
        colorBy="tag",
        colorByTag= "HC",
        sortByTag= "RG",
        visibilityWindow= 1000
)
t3 = igv.create_track(
        name= "mother HC VCF",
        url= "gs://bucket-"+user_name+"/motherHC.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/motherHC.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)

igv.load_track(t2)
igv.load_track(t3)
igv.search_locus("20", 10002294, 10002623)

Czerwone odczyty nie są prawdziwe - (spójrz na RG). Są to odczyty reprezentujące haplotypy utworzone przez HaplotypeCaller. Co jeżeli pokolorujemy po tagu HC?
<div class="alert alert-block alert-warning">
<b>Zadanie 7:</b> Podejrzyj wsparcie dla każdego haplotypu poprzez pokolorowanie po tag'u HC zamiast RG. HaplotypeCaller oznacza odpowiednim tagiem HC wszystkie odczyty jednoznacznie wskazujący na dany "sztuczny haplotyp". Co teraz sądzisz o decyzji Variant Caller'a w sprawie wyboru typu insercji?
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 8:</b> Oddal widok i zobacz ile różnych haplotypów zostało znalezionych dla kolejnych trzech spójnych regionów. Dlaczego w regionie po lewej jest więcej haplotypów?
</div>

## Jednoczesne genotypowanie wielu próbek przy wykorzystaniu gVCF

Istnieje możliwość jednoczesnego genotypowania wielu probek w normalnym trybie pracy działania HaplotypeCallera (joint calling). Jednakże nie jest to rozwiązanie skalowalne. Dlatego rozdzielono przetwarzanie plików BAM od właściwego genotypowania poprzez wprowadzenie tzw. plików gVCF. 

W trybie potoku GVCF HaplotypeCaller urachmiany jest z opcją -ERC GVCF dla każdego pliku BAM, co w wyniku generuje pliki z rozszerzeniem .gvcf posiadające poza informacją o wariantach dodatkowe dane o jakości pozostałych regionów. Następnie jednoczesne genotypowanie wielu próbek odbywa się poprzez wykonanie metody GenotypeGVCF na wielu plikach gvcf. 

Utwórzmy plik g.vcf dla jednego pliku bam (mother.bam).

In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam \
    -O ${WORKSPACE}/sandbox/motherHC.g.vcf \
    -ERC GVCF \
    -L 20:10,000,000-10,200,000

In [ ]:
! bgzip ${WORKSPACE}/sandbox/motherHC.g.vcf
! tabix ${WORKSPACE}/sandbox/motherHC.g.vcf.gz

In [ ]:
! gsutil cp sandbox/motherHC.g.vcf* gs://bucket-$USER/

igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)


t2 = igv.create_track(
        name= "mother HC debug BAM",
        url= "gs://bucket-"+user_name+"/motherHCdebug.bam",
        index_url= "gs://bucket-"+user_name+"/motherHCdebug.bai",
        indexed= True,
        showSoftClips=True,
        colorBy="tag",
        colorByTag= "HC",
        sortByTag= "RG",
        visibilityWindow= 1000
)
t3 = igv.create_track(
        name= "mother HC VCF",
        url= "gs://bucket-"+user_name+"/motherHC.g.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/motherHC.g.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)
igv.load_track(t2)

igv.load_track(t3)
igv.search_locus("20", 10002294, 10002623)

Co sie zmienilo w stosunku do VCF?
Poza wariantami pojawily sie szare bloki reprezentujace stopien pewnosci, ze w danym regionie nie ma wariantu. 
Zmiana bloku oznacza zmiane pewnosci, ze w danym regionie nie ma wariantow. 


In [ ]:
# Podejrzyj kilka wierszy g.vcf
! zcat ${WORKSPACE}/sandbox/motherHC.g.vcf.gz |  grep 10002458

Jezeli podejrzymy zawartosc gVCF, to zobaczymy ze w kolumnie ALT pojawia sie pozycja <NON_REF>.
Identyfikator PL w kolumnie format oznacza prawdopodobienstwo wystepowania kolejnych alleli, wlaczajac <NON_REF>. 


In [ ]:
! rm -r ${WORKSPACE}/sandbox_trio
! mkdir ${WORKSPACE}/sandbox_trio/
! gatk GenomicsDBImport \
    -V ${WORKSPACE}/gvcf/mother.g.vcf.gz \
    -V ${WORKSPACE}/gvcf/father.g.vcf.gz \
    -V ${WORKSPACE}/gvcf/son.g.vcf.gz \
    --genomicsdb-workspace-path ${WORKSPACE}/sandbox_trio/trio \
    --intervals 20:10,000,000-10,200,000

In [ ]:
! ln -s ${WORKSPACE}/sandbox_trio/ sandbox_trio

In [ ]:
# z bazy mozemy wyciagnac polaczony gvcf
! gatk SelectVariants \
    -R ${WORKSPACE}/ref/ref.fasta \
    -V gendb://sandbox_trio/trio \
    -O ${WORKSPACE}/sandbox/trio_selectvariants.g.vcf

In [ ]:
! cat ${WORKSPACE}/sandbox/trio_selectvariants.g.vcf | grep 10002458

In [ ]:
Ostatnim krokiem jest genotyopwanie

In [ ]:
! gatk GenotypeGVCFs \
    -R ${WORKSPACE}/ref/ref.fasta \
    -V gendb://sandbox_trio/trio \
    -O ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -L 20:10,000,000-10,200,000

In [ ]:
! bgzip -f ${WORKSPACE}/sandbox/trioGGVCF.vcf 
! tabix ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz
! gsutil cp sandbox/trioGGVCF.vcf.gz* gs://bucket-$USER/

In [ ]:
from igv_jupyterlab import IGV
igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)


t2 = igv.create_track(
        name= "mother HC debug BAM",
        url= "gs://bucket-"+user_name+"/motherHCdebug.bam",
        index_url= "gs://bucket-"+user_name+"/motherHCdebug.bai",
        indexed= True,
        showSoftClips=True,
        colorBy="tag",
        colorByTag= "HC",
        sortByTag= "RG",
        visibilityWindow= 1000
)
t3 = igv.create_track(
        name= "trio VCF",
        url= "gs://bucket-"+user_name+"/trioGGVCF.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/trioGGVCF.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)
#igv.load_track(t2)

igv.load_track(t3)

igv.search_locus("20", 10002294, 10002623)

# Poprawa genotypowania

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://us.v-cdn.net/5019796/uploads/FileUpload/40/a04b64d7884a07d1b562ead4002c60.jpg")

In [ ]:
! gunzip -k ${WORKSPACE}/sandbox/trioGGVCF.vcf 
! gatk CalculateGenotypePosteriors \
    -V ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -ped ${WORKSPACE}/trio.ped \
    --skip-population-priors \
    -O ${WORKSPACE}/sandbox/trioCGP.vcf

In [ ]:
! gatk CalculateGenotypePosteriors \
    -V ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -ped ${WORKSPACE}/trio.ped \
    --supporting-callsets ${WORKSPACE}/resources/af-only-gnomad.chr20subset.b37.vcf.gz \
    -O ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf

In [ ]:
! bgzip -f  ${WORKSPACE}/sandbox/trioGGVCF.vcf
! tabix -f ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz
! bgzip -f  ${WORKSPACE}/sandbox/trioCGP.vcf
! tabix -f ${WORKSPACE}/sandbox/trioCGP.vcf.gz
! bgzip -f  ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf
! tabix -f ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf.gz
! gsutil cp -m ${WORKSPACE}/sandbox/trio* gs://bucket-$USER/


In [ ]:
!gsutil ls "gs://bucket-"$USER

In [ ]:
from igv_jupyterlab import IGV
igv = IGV(genome="hg19" )
display(igv)
import time
time.sleep(3)


t1 = igv.create_track(
        name= "trio GVCF",
        url= "gs://bucket-"+user_name+"/trioGGVCF.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/trioGGVCF.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)

t2 = igv.create_track(
        name= "trio CGP",
        url= "gs://bucket-"+user_name+"/trioCGP.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/trioCGP.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)


t3 = igv.create_track(
        name= "trio CGP gnomad",
        url= "gs://bucket-"+user_name+"/trioCGP_gnomad.vcf.gz",
        index_url= "gs://bucket-"+user_name+"/trioCGP_gnomad.vcf.gz.tbi",
        indexed= True,
        visibilityWindow= 1000
)


igv.load_track(t1)
igv.load_track(t2)
igv.load_track(t3)

igv.search_locus("20", 10002294, 10002623)


CalculateGenotypePosteriors dodaje 3 pozycje do kolumny FORMAT: PP, JL i JP.

- Phred-scaled Posterior Probability (PP) basically refines the PL values. It incorporates the prior expectations for the given pedigree and/or population allele frequencies.
- Joint Trio Likelihood (JL) is the Phred-scaled joint likelihood of the posterior genotypes for the trio being incorrect.
- Joint Trio Posterior (JP) is the Phred-scaled posterior probability of the posterior genotypes for the three samples being incorrect.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/gatk-tutorials/workshop_1906/2-germline/images/vd-image4.png")


In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/gatk-tutorials/workshop_1906/2-germline/images/vd-image5.png")


Więcej szczegółów na https://gatk.broadinstitute.org/hc/en-us/articles/360035531432?id=11074

# Analiza pokrycia

In [ ]:
import os
import pysam
counter = 0
with pysam.AlignmentFile(workspace+'/bam/mother.bam', 'rb') as samfile:
    for pileupcolumn in samfile.pileup("20", 10002294, 10002303):
        counter +=1;
        #if (pileupcolumn.pos != 16099988):   continue;
        print ("\ncoverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
        if (counter > 10): break

In [ ]:
! samtools depth -a -r 20:10002294-10002623   ${WORKSPACE}/bam/mother.bam -o ${WORKSPACE}/sandbox/mother_cov

In [ ]:
! head  ${WORKSPACE}/sandbox/mother_cov

In [ ]:
import pandas as pd
tsv_read = pd.read_csv(workspace+'/sandbox/mother_cov', sep='\t' , header=None)

In [ ]:
tsv_read.columns = ["chr", "start", "coverage"]
tsv_read

<div class="alert alert-block alert-warning">
<b>Zadanie 9:</b> Zrób wykres głębokości pokrycia dla pozycji (chr20:10002294-10002623)i porownaj go z wizualizacja w IGV
</div>

In [ ]:
# %lmagic ex9
# Solution to ex9

import matplotlib.pyplot  as plt
idx = (tsv_read.start > 10002294) & (tsv_read.start < 10002623)
plt.plot(tsv_read.start[idx], tsv_read.coverage[idx] , linewidth=15.0)
plt.axis([None, None, 0, 40])
#fig=plt.figure(figsize=(20, 10))


# Adnotacje

In [ ]:
from biothings_client import get_client
import pandas as pd
pd.set_option('display.max_columns', 2000)
mv = get_client('variant')
mv.getvariants(["chr7:g.140453134T>C","chr7:g.140453134T>C"], as_dataframe=1)

In [ ]:
list(mv.query('dbnsfp.genename:CDK2', fetch_all=True))[0]
#http://docs.myvariant.info/en/latest/doc/data.html#available-fields

In [ ]:
#mv.getvariants('chr17:g.7578532A>G', fields = ['clinvar','gnomad_af'] , as_dataframe=1)
gg = mv.getvariants('chr7:g.140453134T>C', fields = "all" , as_dataframe=1)
gg.shape
gg

#https://cdn.rawgit.com/biothings/myvariant.info/master/docs/ipynb/myvariant_clinvar_demo.html

In [ ]:
! gunzip -k ${WORKSPACE}/sandbox/motherHC.vcf
df = mv.getvariants(mv.get_hgvs_from_vcf(workspace + "/sandbox/motherHC.vcf") \
                    ,fields="all", as_dataframe=1, fetch_all=True)


In [ ]:
df[1:10]

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns

In [ ]:
df["gnomad_genome.af.af_popmax"].plot(figsize=(20,10))

In [ ]:
df.groupby(['dbsnp.gene.name']).count()

# Ocena jakości wariantów przy użyciu CollectVariantCallingMetrics
 
CollectVariantCallingMetrics wylicza zbiorecze metryki na poziomie probki i genotypu. 

In [ ]:
! gatk CollectVariantCallingMetrics \
    -I  ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz \
    --DBSNP ${WORKSPACE}/resources/dbsnp.vcf \
    -O ${WORKSPACE}/sandbox/trioGGVCF_metrics